In [4]:
import pandas as pd
from utils.load_data import Dataload
from pathlib import Path
from utils.ressources import *
from utils.load_and_apply_model import *
import warnings

# Désactiver tous les avertissements
warnings.filterwarnings("ignore")

current_path = Path.cwd()
df_dataviz = current_path / "data" / "df_dataviz.csv"
df_prepared = current_path / "data" / "data_features_city.csv"


path_df = current_path / "data" / "data_features.csv"

df_dataviz = Dataload(df_dataviz).load_df()
# print(df.head())


path_df = current_path / "data" / "data_features.csv"
path_model = current_path / "models"


C:\Users\benme\Documents\virtualenv\venv_weatherforecast\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\benme\Documents\virtualenv\venv_weatherforecast\lib\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
2023-06-20 23:18:33.085 INFO    visions.backends: Pandas backend loaded 1.5.2
2023-06

In [2]:

X_train, X_test, y_train, y_test = Dataload(path_df).split_data_train_test()

print(X_test.iloc[:, :-1].columns)
print(X_test.iloc[:, :-1].shape)

Index(['sunshine', 'windgustspeed', 'humidity3pm', 'pressure3pm', 'year',
       'windspeed3pm', 'temp_amplitude', 'cloud3pm', 'rainfall', 'climat',
       'humidity9am', 'winddir3pm_sin', 'longitude', 'winddir9am_cos',
       'cloud9am', 'winddir9am_sin', 'latitude', 'month', 'geography',
       'raintoday', 'evaporation', 'windspeed9am'],
      dtype='object')
(14542, 22)


In [3]:
X_test

,sunshine,windgustspeed,humidity3pm,pressure3pm,year,windspeed3pm,temp_amplitude,cloud3pm,rainfall,climat,...,winddir9am_cos,cloud9am,winddir9am_sin,latitude,month,geography,raintoday,evaporation,windspeed9am,windgustdir_cos
45383,11.7,28.0,62.0,1004.5,2015,15.0,7.2,3.0,0.6,0,...,-7.071068e-01,4.0,7.071068e-01,-16.595325,1,1,0,5.8,4.0,6.123234e-17
44335,10.0,35.0,54.0,1012.5,2010,20.0,7.4,1.0,0.6,0,...,3.826834e-01,5.0,-9.238795e-01,-16.595325,9,1,0,3.6,13.0,9.238795e-01
68858,10.4,48.0,20.0,1025.9,2014,26.0,20.5,1.0,0.0,1,...,-3.826834e-01,2.0,-9.238795e-01,-23.215601,8,2,0,7.0,9.0,3.826834e-01
32339,9.9,54.0,32.0,1008.1,2017,15.0,14.3,1.0,0.0,2,...,-9.238795e-01,7.0,3.826834e-01,-37.667395,2,0,0,7.4,13.0,-3.826834e-01
12265,1.0,43.0,66.0,1003.5,2012,20.0,6.4,7.0,0.0,0,...,-1.000000e+00,7.0,1.224647e-16,-33.758057,1,1,0,8.8,7.0,7.071068e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11013,0.3,22.0,69.0,1015.2,2012,13.0,4.2,7.0,0.0,0,...,-3.826834e-01,7.0,-9.238795e-01,-28.945667,7,3,0,2.6,11.0,-9.238795e-01
59477,6.7,37.0,20.0,1019.0,2012,13.0,12.6,5.0,0.0,3,...,9.238795e-01,6.0,3.826834e-01,-31.938420,4,1,0,4.6,20.0,7.071068e-01
70885,9.6,37.0,45.0,1008.7,2015,20.0,8.7,1.0,0.0,0,...,1.000000e+00,7.0,0.000000e+00,-12.457743,4,1,0,6.8,19.0,3.826834e-01
60564,5.8,30.0,69.0,1020.2,2016,15.0,13.1,5.0,0.0,3,...,6.123234e-17,2.0,1.000000e+00,-31.938420,5,1,0,3.6,11.0,-9.238795e-01


In [6]:
Dataload(path_df).load_df().columns

Index(['sunshine', 'windgustspeed', 'humidity3pm', 'pressure3pm', 'year',
       'windspeed3pm', 'temp_amplitude', 'cloud3pm', 'rainfall', 'climat',
       'humidity9am', 'winddir3pm_sin', 'longitude', 'winddir9am_cos',
       'cloud9am', 'winddir9am_sin', 'latitude', 'month', 'geography',
       'raintoday', 'evaporation', 'windspeed9am', 'windgustdir_cos',
       'raintomorrow'],
      dtype='object')

In [10]:
df_dataviz

,date,location,mintemp,maxtemp,rainfall,evaporation,sunshine,windgustdir,windgustspeed,winddir9am,...,latitude,longitude,climat_koppen,year,month,day,climat,geography,position,temp_amplitude
0,2008-12-01,Albury,13.4,22.9,0.6,7.6,10.9,W,44.0,W,...,-36.065766,146.910315,Cfa,2008,12,1,Chaud humide,valley,S,9.5
1,2008-12-02,Albury,7.4,25.1,0.0,12.0,12.9,WNW,44.0,NNW,...,-36.065766,146.910315,Cfa,2008,12,2,Chaud humide,valley,S,17.7
2,2008-12-03,Albury,12.9,25.7,0.0,8.0,8.0,WSW,46.0,W,...,-36.065766,146.910315,Cfa,2008,12,3,Chaud humide,valley,S,12.8
3,2008-12-04,Albury,9.2,28.0,0.0,10.0,2.2,NE,24.0,SE,...,-36.065766,146.910315,Cfa,2008,12,4,Chaud humide,valley,S,18.8
4,2008-12-05,Albury,17.5,32.3,1.0,4.8,6.7,W,41.0,ENE,...,-36.065766,146.910315,Cfa,2008,12,5,Chaud humide,valley,S,14.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145028,2016-04-20,Uluru,18.8,31.7,0.0,8.0,9.9,SSW,31.0,S,...,-24.997901,131.089241,BSh,2016,4,20,Sec,desert,S,12.9
145031,2016-04-23,Uluru,16.9,28.4,0.0,18.4,11.0,E,31.0,ESE,...,-24.997901,131.089241,BSh,2016,4,23,Sec,desert,S,11.5
145032,2016-04-24,Uluru,12.1,27.3,0.0,8.2,11.0,E,31.0,ENE,...,-24.997901,131.089241,BSh,2016,4,24,Sec,desert,S,15.2
145033,2016-04-25,Uluru,9.6,28.8,0.0,7.8,11.0,E,30.0,E,...,-24.997901,131.089241,BSh,2016,4,25,Sec,desert,S,19.2


In [17]:
date_du_jour = pd.to_datetime('2015-04-15')

In [23]:
df_dataviz['date'] = pd.to_datetime(df_dataviz['date'])

df_selection = df_dataviz[df_dataviz['date'] == date_du_jour]
df_selection

,date,location,mintemp,maxtemp,rainfall,evaporation,sunshine,windgustdir,windgustspeed,winddir9am,...,latitude,longitude,climat_koppen,year,month,day,climat,geography,position,temp_amplitude
2237,2015-04-15,Albury,12.7,24.2,3.0,4.0,5.4,ENE,39.0,ENE,...,-36.065766,146.910315,Cfa,2015,4,15,Chaud humide,valley,S,11.5
11264,2015-04-15,Coffs Harbour,15.3,25.6,0.0,3.0,10.2,NE,31.0,NNW,...,-30.234976,153.099084,Cfa,2015,4,15,Chaud humide,coastal,S,10.3
14273,2015-04-15,Moree,16.6,28.9,0.0,6.0,10.9,E,35.0,NNE,...,-29.447084,149.839583,Cfa,2015,4,15,Chaud humide,valley,N,12.3
20316,2015-04-15,Norah Head,16.2,25.7,0.0,5.0,8.7,NNW,52.0,N,...,-33.275517,151.566867,Cfa,2015,4,15,Chaud humide,coastal,S,9.5
23325,2015-04-15,Norfolk Island,17.8,22.4,0.4,4.2,5.3,S,35.0,SSE,...,-28.945667,167.978584,Cfa,2015,4,15,Chaud humide,island,E,4.6
32717,2015-04-15,Sydney,15.0,24.2,0.0,5.0,8.7,W,46.0,W,...,-33.758057,151.149932,Cfa,2015,4,15,Chaud humide,coastal,E,9.2
35726,2015-04-15,Sydney Airport,15.2,26.7,0.0,5.0,8.7,W,46.0,NW,...,-33.947598,151.182616,Cfa,2015,4,15,Chaud humide,coastal,E,11.5
38735,2015-04-15,Wagga Wagga,14.3,25.1,0.0,4.0,5.4,SSW,41.0,ENE,...,-34.536641,147.104610,Cfa,2015,4,15,Chaud humide,valley,S,10.8
41744,2015-04-15,Williamtown,9.3,27.3,0.0,3.0,10.2,SW,39.0,NW,...,-32.188655,151.835389,Cfa,2015,4,15,Chaud humide,valley,S,18.0
48220,2015-04-15,Canberra,7.0,21.9,0.0,4.4,9.3,W,26.0,NNW,...,-35.137242,149.103799,Cfb,2015,4,15,Tempéré froid,mountainous,S,14.9


In [22]:
df_prepared

WindowsPath('C:/Users/benme/Documents/datascientest/projet/australia_weather_forecasts/webapp/data/data_features_city.csv')

In [25]:
df_prepared = pd.read_csv(df_prepared, index_col=0)
df_prepared['date'] = pd.to_datetime(df_prepared['date'])

df_selection_df_prepared = df_prepared[df_prepared['date'] == date_du_jour]
df_selection_df_prepared

KeyError: 'date'

In [26]:
df_prepared

,location,evaporation,sunshine,windgustspeed,windspeed9am,windspeed3pm,humidity9am,humidity3pm,pressure3pm,cloud9am,...,climat,geography,position,temp_amplitude,winddir9am_sin,winddir9am_cos,winddir3pm_sin,winddir3pm_cos,windgustdir_sin,windgustdir_cos
0,Albury,7.6,10.9,44.0,20.0,24.0,71.0,22.0,1007.1,8.0,...,0,0,0,9.5,1.224647e-16,-1.000000e+00,3.826834e-01,-0.923880,1.224647e-16,-1.000000e+00
1,Albury,12.0,12.9,44.0,4.0,22.0,44.0,25.0,1007.8,1.0,...,0,0,0,17.7,9.238795e-01,-3.826834e-01,-3.826834e-01,-0.923880,3.826834e-01,-9.238795e-01
2,Albury,8.0,8.0,46.0,19.0,26.0,38.0,30.0,1008.7,7.0,...,0,0,0,12.8,1.224647e-16,-1.000000e+00,-3.826834e-01,-0.923880,-3.826834e-01,-9.238795e-01
3,Albury,10.0,2.2,24.0,11.0,9.0,45.0,16.0,1012.8,8.0,...,0,0,0,18.8,-7.071068e-01,7.071068e-01,0.000000e+00,1.000000,7.071068e-01,7.071068e-01
4,Albury,4.8,6.7,41.0,7.0,20.0,82.0,33.0,1006.0,7.0,...,0,0,0,14.8,3.826834e-01,9.238795e-01,7.071068e-01,-0.707107,1.224647e-16,-1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145027,Uluru,9.6,10.9,31.0,9.0,11.0,30.0,12.0,1010.3,3.0,...,1,2,0,21.7,0.000000e+00,1.000000e+00,1.224647e-16,-1.000000,-3.826834e-01,-9.238795e-01
145028,Uluru,8.0,9.9,31.0,17.0,19.0,44.0,24.0,1013.8,8.0,...,1,2,0,12.9,-1.000000e+00,-1.836970e-16,7.071068e-01,0.707107,-9.238795e-01,-3.826834e-01
145032,Uluru,8.2,11.0,31.0,22.0,19.0,42.0,21.0,1017.9,0.0,...,1,2,0,15.2,3.826834e-01,9.238795e-01,0.000000e+00,1.000000,0.000000e+00,1.000000e+00
145033,Uluru,7.8,11.0,30.0,13.0,9.0,28.0,13.0,1017.1,0.0,...,1,2,0,19.2,0.000000e+00,1.000000e+00,9.238795e-01,0.382683,0.000000e+00,1.000000e+00


In [30]:
import pandas as pd

def filter_by_date(df, date_du_jour):
    # Convertir les colonnes "annee", "mois" et "jour" en type entier
    df['year'] = df['year'].astype(int)
    df['month'] = df['month'].astype(int)
    df['day'] = df['day'].astype(int)

    df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
    date_du_jour = pd.to_datetime(date_du_jour)
    df_filtered = df[df['date'] == date_du_jour]

    return df_filtered


In [32]:
# Appeler la fonction filter_by_date pour filtrer le DataFrame à partir de la date souhaitée
df_filtered = filter_by_date(df_prepared, "27/05/2015")

# Afficher le DataFrame filtré
df_filtered

,location,evaporation,sunshine,windgustspeed,windspeed9am,windspeed3pm,humidity9am,humidity3pm,pressure3pm,cloud9am,...,geography,position,temp_amplitude,winddir9am_sin,winddir9am_cos,winddir3pm_sin,winddir3pm_cos,windgustdir_sin,windgustdir_cos,date
5288,Badgerys Creek,3.2,8.3,13.0,6.0,7.0,82.0,50.0,1021.8,6.0,...,1,1,14.0,-9.238795e-01,-3.826834e-01,1.000000e+00,6.123234e-17,1.000000e+00,6.123234e-17,2015-05-27
11306,Coffs Harbour,2.2,5.1,30.0,7.0,19.0,81.0,63.0,1021.8,5.0,...,1,0,13.0,7.071068e-01,-7.071068e-01,7.071068e-01,7.071068e-01,7.071068e-01,7.071068e-01,2015-05-27
14315,Moree,1.2,9.0,28.0,17.0,9.0,80.0,43.0,1022.2,3.0,...,0,2,14.6,0.000000e+00,1.000000e+00,-9.238795e-01,3.826834e-01,0.000000e+00,1.000000e+00,2015-05-27
20358,Norah Head,3.2,8.3,20.0,4.0,7.0,71.0,61.0,1021.9,6.0,...,1,0,8.6,3.826834e-01,-9.238795e-01,7.071068e-01,-7.071068e-01,1.224647e-16,-1.000000e+00,2015-05-27
23367,Norfolk Island,3.0,4.4,35.0,11.0,17.0,57.0,52.0,1020.7,7.0,...,3,1,6.4,3.826834e-01,-9.238795e-01,1.224647e-16,-1.000000e+00,-7.071068e-01,-7.071068e-01,2015-05-27
32759,Sydney,3.2,8.3,26.0,15.0,9.0,69.0,40.0,1021.6,6.0,...,1,1,11.2,1.224647e-16,-1.000000e+00,3.826834e-01,-9.238795e-01,1.224647e-16,-1.000000e+00,2015-05-27
35768,Sydney Airport,3.2,8.3,26.0,15.0,13.0,60.0,38.0,1021.6,6.0,...,1,1,11.0,7.071068e-01,-7.071068e-01,1.224647e-16,-1.000000e+00,7.071068e-01,-7.071068e-01,2015-05-27
38777,Wagga Wagga,0.8,1.1,22.0,15.0,11.0,88.0,84.0,1022.5,7.0,...,0,0,7.2,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,7.071068e-01,7.071068e-01,2015-05-27
41786,Williamtown,2.2,5.1,30.0,20.0,17.0,77.0,52.0,1022.4,5.0,...,0,0,9.9,7.071068e-01,-7.071068e-01,3.826834e-01,-9.238795e-01,7.071068e-01,-7.071068e-01,2015-05-27
48262,Canberra,3.2,6.5,33.0,7.0,24.0,85.0,61.0,1022.6,8.0,...,4,0,17.1,-3.826834e-01,9.238795e-01,3.826834e-01,-9.238795e-01,3.826834e-01,-9.238795e-01,2015-05-27


In [40]:
import pandas as pd
import plotly.express as px

def display_map_rain(df):
    colors_rain = ['red', 'blue']  # Couleurs pour les jours sans pluie et avec pluie

    fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='raintomorrow', color_discrete_sequence=colors_rain,
                            title='Prévision de pluie du lendemain', hover_name='location', hover_data=['raintomorrow'])

    # Spécifier l'ordre des catégories pour la légende
    fig.update_layout(coloraxis_colorbar=dict(title="Prévision de pluie du lendemain", tickvals=[0, 1], ticktext=['Non', 'Oui']))

    # Référence de tuile
    fig.update_layout(mapbox_style='carto-positron')

    # Limites de la carte et le zoom de départ
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0}, mapbox_center_lat=-25, mapbox_center_lon=135,
                      mapbox_zoom=2)

    # Augmentation de la taille des points
    fig.update_traces(marker=dict(size=10))

    # Afficher le graphique dans le notebook
    fig.show()


In [42]:
filter_by_date

<function __main__.filter_by_date(df, date_du_jour)>

In [44]:
filter_by_date(df_prepared, "27/08/2015")

,location,evaporation,sunshine,windgustspeed,windspeed9am,windspeed3pm,humidity9am,humidity3pm,pressure3pm,cloud9am,...,geography,position,temp_amplitude,winddir9am_sin,winddir9am_cos,winddir3pm_sin,winddir3pm_cos,windgustdir_sin,windgustdir_cos,date
14407,Moree,2.8,0.3,17.0,13.0,7.0,89.0,90.0,1020.9,8.0,...,0,2,3.8,9.238795e-01,0.382683,0.382683,9.238795e-01,3.826834e-01,9.238795e-01,2015-08-27
32851,Sydney,3.0,4.3,52.0,9.0,17.0,87.0,44.0,1015.1,7.0,...,1,1,8.9,1.224647e-16,-1.000000,0.707107,-7.071068e-01,1.224647e-16,-1.000000e+00,2015-08-27
35860,Sydney Airport,3.0,4.3,52.0,13.0,22.0,68.0,38.0,1014.9,7.0,...,1,1,10.8,3.826834e-01,-0.923880,0.707107,-7.071068e-01,1.224647e-16,-1.000000e+00,2015-08-27
41878,Williamtown,3.4,4.3,52.0,17.0,30.0,71.0,52.0,1017.5,7.0,...,0,0,6.0,7.071068e-01,-0.707107,0.382683,-9.238795e-01,7.071068e-01,-7.071068e-01,2015-08-27
51393,Tuggeranong,1.2,0.0,33.0,7.0,15.0,99.0,59.0,1016.1,7.0,...,4,0,6.0,-3.826834e-01,-0.923880,0.707107,-7.071068e-01,7.071068e-01,-7.071068e-01,2015-08-27
57473,Ballarat,1.2,0.0,39.0,7.0,19.0,100.0,75.0,1020.8,8.0,...,0,0,11.5,-9.238795e-01,-0.382683,-0.707107,-7.071068e-01,-1.000000e+00,-1.836970e-16,2015-08-27
60513,Bendigo,1.2,0.0,39.0,9.0,26.0,84.0,52.0,1019.6,1.0,...,0,0,10.1,-7.071068e-01,-0.707107,-0.923880,-3.826834e-01,-1.000000e+00,-1.836970e-16,2015-08-27
66531,Melbourne Airport,1.2,0.0,44.0,11.0,13.0,93.0,92.0,1020.2,7.0,...,0,0,3.1,-3.826834e-01,-0.923880,-0.923880,-3.826834e-01,-1.000000e+00,-1.836970e-16,2015-08-27
72733,Mildura,2.4,8.4,35.0,9.0,15.0,84.0,33.0,1022.2,7.0,...,2,0,10.3,-7.071068e-01,-0.707107,-0.923880,-3.826834e-01,-9.238795e-01,-3.826834e-01,2015-08-27
77320,Portland,1.8,5.2,37.0,4.0,22.0,98.0,91.0,1023.1,1.0,...,1,0,9.1,-3.826834e-01,-0.923880,-1.000000,-1.836970e-16,-1.000000e+00,-1.836970e-16,2015-08-27


In [46]:
display_map_rain(filter_by_date(df_prepared, "27/08/2015"))

In [48]:
df_prepared = filter_by_date(df_prepared, "27/08/2015")

In [49]:
import folium

def display_map_folium(df):
    # Création de la carte centrée sur l'Australie
    map = folium.Map(location=[-25, 135], zoom_start=4)

    # Parcourir chaque ligne du DataFrame
    for index, row in df.iterrows():
        # Obtenir les coordonnées de la localisation
        latitude = row['latitude']
        longitude = row['longitude']

        # Obtenir la valeur de raintomorrow
        raintomorrow = row['raintomorrow']

        # Déterminer la couleur du marqueur en fonction de la valeur de raintomorrow
        color = 'blue' if raintomorrow == 1 else 'red'

        # Créer le marqueur et l'ajouter à la carte
        folium.Marker(location=[latitude, longitude], icon=folium.Icon(color=color)).add_to(map)

    # Afficher la carte
    return map

# Utilisation de la fonction pour afficher la carte
map = display_map_folium(df_prepared)
map


In [5]:
df_city = current_path / "data" / "data_features_city.csv"
df_city = Dataload(df_city).load_df()


In [6]:
df_city.head()

,location,evaporation,sunshine,windgustspeed,windspeed9am,windspeed3pm,humidity9am,humidity3pm,pressure3pm,cloud9am,...,climat,geography,position,temp_amplitude,winddir9am_sin,winddir9am_cos,winddir3pm_sin,winddir3pm_cos,windgustdir_sin,windgustdir_cos
0,Albury,7.6,10.9,44.0,20.0,24.0,71.0,22.0,1007.1,8.0,...,0,0,0,9.5,1.224647e-16,-1.000000,0.382683,-0.923880,1.224647e-16,-1.000000
1,Albury,12.0,12.9,44.0,4.0,22.0,44.0,25.0,1007.8,1.0,...,0,0,0,17.7,9.238795e-01,-0.382683,-0.382683,-0.923880,3.826834e-01,-0.923880
2,Albury,8.0,8.0,46.0,19.0,26.0,38.0,30.0,1008.7,7.0,...,0,0,0,12.8,1.224647e-16,-1.000000,-0.382683,-0.923880,-3.826834e-01,-0.923880
3,Albury,10.0,2.2,24.0,11.0,9.0,45.0,16.0,1012.8,8.0,...,0,0,0,18.8,-7.071068e-01,0.707107,0.000000,1.000000,7.071068e-01,0.707107
4,Albury,4.8,6.7,41.0,7.0,20.0,82.0,33.0,1006.0,7.0,...,0,0,0,14.8,3.826834e-01,0.923880,0.707107,-0.707107,1.224647e-16,-1.000000


In [7]:
from utils.load_and_apply_model import *
from utils.load_data import filter_by_date

In [19]:
df2 = filter_by_date(df_city,"01/12/2008", return_index=True)

In [20]:
df2

Int64Index([0, 49023, 58142, 67353, 84160, 87200, 90240, 120791, 129991,
            139264],
           dtype='int64')

In [9]:
model_random_forest = load_model(path_model / "randomforestclassifier_model.joblib")

Model loaded


In [21]:
list(df2)

[0, 49023, 58142, 67353, 84160, 87200, 90240, 120791, 129991, 139264]

In [12]:
df_data_features = current_path / "data" / "data_features.csv"
df_data_features = Dataload(df_data_features).load_df()

In [13]:
df_data_features

,sunshine,windgustspeed,humidity3pm,pressure3pm,year,windspeed3pm,temp_amplitude,cloud3pm,rainfall,climat,...,cloud9am,winddir9am_sin,latitude,month,geography,raintoday,evaporation,windspeed9am,windgustdir_cos,raintomorrow
0,10.9,44.0,22.0,1007.1,2008,24.0,9.5,7.0,0.6,0,...,8.0,1.224647e-16,-36.065766,12,0,0,7.6,20.0,-1.000000e+00,0
1,12.9,44.0,25.0,1007.8,2008,22.0,17.7,1.0,0.0,0,...,1.0,9.238795e-01,-36.065766,12,0,0,12.0,4.0,-9.238795e-01,0
2,8.0,46.0,30.0,1008.7,2008,26.0,12.8,2.0,0.0,0,...,7.0,1.224647e-16,-36.065766,12,0,0,8.0,19.0,-9.238795e-01,0
3,2.2,24.0,16.0,1012.8,2008,9.0,18.8,6.0,0.0,0,...,8.0,-7.071068e-01,-36.065766,12,0,0,10.0,11.0,7.071068e-01,0
4,6.7,41.0,33.0,1006.0,2008,20.0,14.8,8.0,1.0,0,...,7.0,3.826834e-01,-36.065766,12,0,0,4.8,7.0,-1.000000e+00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72702,10.9,31.0,12.0,1010.3,2016,11.0,21.7,4.0,0.0,1,...,3.0,0.000000e+00,-24.997901,4,2,0,9.6,9.0,-9.238795e-01,0
72703,9.9,31.0,24.0,1013.8,2016,19.0,12.9,1.0,0.0,1,...,8.0,-1.000000e+00,-24.997901,4,2,0,8.0,17.0,-3.826834e-01,0
72704,11.0,31.0,21.0,1017.9,2016,19.0,15.2,0.0,0.0,1,...,0.0,3.826834e-01,-24.997901,4,2,0,8.2,22.0,1.000000e+00,0
72705,11.0,30.0,13.0,1017.1,2016,9.0,19.2,0.0,0.0,1,...,0.0,0.000000e+00,-24.997901,4,2,0,7.8,13.0,1.000000e+00,0


In [22]:
df_data_features.loc[df2]

KeyError: '[84160, 87200, 90240, 120791, 129991, 139264] not in index'

In [24]:
df_data_features.shape

(72707, 24)

In [34]:
df_data_features[['year', 'month']]

,year,month
0,2008,12
1,2008,12
2,2008,12
3,2008,12
4,2008,12
...,...,...
72702,2016,4
72703,2016,4
72704,2016,4
72705,2016,4


In [46]:
df_city['day'].info()

<class 'pandas.core.series.Series'>
Int64Index: 72707 entries, 0 to 145034
Series name: day
Non-Null Count  Dtype
--------------  -----
72707 non-null  int32
dtypes: int32(1)
memory usage: 2.8 MB


In [47]:
df_data_features['day'] =  df_city['day'].values

In [48]:
df_data_features_webapp = df_data_features[['sunshine', 'windgustspeed', 'humidity3pm', 'pressure3pm', 'year','day' , 'month',
       'windspeed3pm', 'temp_amplitude', 'cloud3pm', 'rainfall', 'climat',
       'humidity9am', 'winddir3pm_sin', 'longitude', 'winddir9am_cos',
       'cloud9am', 'winddir9am_sin', 'latitude',  'geography',
       'raintoday', 'evaporation', 'windspeed9am', 'windgustdir_cos',
       'raintomorrow']]

In [50]:
df_data_features_webapp.to_csv(current_path / "data" / "data_features_webapp.csv")

In [53]:
df_data_features_webapp.columns

Index(['sunshine', 'windgustspeed', 'humidity3pm', 'pressure3pm', 'year',
       'day', 'month', 'windspeed3pm', 'temp_amplitude', 'cloud3pm',
       'rainfall', 'climat', 'humidity9am', 'winddir3pm_sin', 'longitude',
       'winddir9am_cos', 'cloud9am', 'winddir9am_sin', 'latitude', 'geography',
       'raintoday', 'evaporation', 'windspeed9am', 'windgustdir_cos',
       'raintomorrow'],
      dtype='object')

In [54]:
df_data_features_webapp

,sunshine,windgustspeed,humidity3pm,pressure3pm,year,day,month,windspeed3pm,temp_amplitude,cloud3pm,...,winddir9am_cos,cloud9am,winddir9am_sin,latitude,geography,raintoday,evaporation,windspeed9am,windgustdir_cos,raintomorrow
0,10.9,44.0,22.0,1007.1,2008,1,12,24.0,9.5,7.0,...,-1.000000e+00,8.0,1.224647e-16,-36.065766,0,0,7.6,20.0,-1.000000e+00,0
1,12.9,44.0,25.0,1007.8,2008,2,12,22.0,17.7,1.0,...,-3.826834e-01,1.0,9.238795e-01,-36.065766,0,0,12.0,4.0,-9.238795e-01,0
2,8.0,46.0,30.0,1008.7,2008,3,12,26.0,12.8,2.0,...,-1.000000e+00,7.0,1.224647e-16,-36.065766,0,0,8.0,19.0,-9.238795e-01,0
3,2.2,24.0,16.0,1012.8,2008,4,12,9.0,18.8,6.0,...,7.071068e-01,8.0,-7.071068e-01,-36.065766,0,0,10.0,11.0,7.071068e-01,0
4,6.7,41.0,33.0,1006.0,2008,5,12,20.0,14.8,8.0,...,9.238795e-01,7.0,3.826834e-01,-36.065766,0,0,4.8,7.0,-1.000000e+00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72702,10.9,31.0,12.0,1010.3,2016,19,4,11.0,21.7,4.0,...,1.000000e+00,3.0,0.000000e+00,-24.997901,2,0,9.6,9.0,-9.238795e-01,0
72703,9.9,31.0,24.0,1013.8,2016,20,4,19.0,12.9,1.0,...,-1.836970e-16,8.0,-1.000000e+00,-24.997901,2,0,8.0,17.0,-3.826834e-01,0
72704,11.0,31.0,21.0,1017.9,2016,24,4,19.0,15.2,0.0,...,9.238795e-01,0.0,3.826834e-01,-24.997901,2,0,8.2,22.0,1.000000e+00,0
72705,11.0,30.0,13.0,1017.1,2016,25,4,9.0,19.2,0.0,...,1.000000e+00,0.0,0.000000e+00,-24.997901,2,0,7.8,13.0,1.000000e+00,0
